In [1]:
import numpy as np
import pandas as pd
import re
import pprint as pp

In [2]:
pdf_path = r"..\data\raw\paypal-transactions.CSV"
df = pd.read_csv(pdf_path)
df.head(5)

,Date,Time,Time Zone,Description,Currency,Gross,Fee,Net,Balance,Transaction ID,From Email Address,Name,Bank Name,Bank Account,Shipping and Handling Amount,Sales Tax,Invoice ID,Reference Txn ID
0,1/6/2020,21:58:32,America/Los_Angeles,PreApproved Payment Bill User Payment,USD,-63.96,0.00,-63.96,-63.96,1LV065757J618651C,paypal_noa@nintendo.co.jp,Nintendo,NaN,NaN,0.0,0.0,BA33852653,B-3NF777293G959551F
1,1/6/2020,21:58:32,America/Los_Angeles,BML Credit - Transfer from BML,USD,63.96,0.00,63.96,0.00,95C42925KP228472V,NaN,NaN,NaN,NaN,0.0,0.0,BA33852653,1LV065757J618651C
2,2/1/2020,00:14:41,America/Los_Angeles,General Payment,USD,10.80,-0.78,10.02,10.02,09R47510L61509220,480553751@qq.com,洁 刘,NaN,NaN,0.0,0.0,NaN,NaN
3,2/11/2020,15:36:27,America/Los_Angeles,General Withdrawal - Bank Account,USD,-10.02,0.00,-10.02,0.00,0MT19863F1612381F,NaN,NaN,PNC,2496.0,0.0,0.0,NaN,NaN
4,2/12/2020,10:30:37,America/Los_Angeles,General Card Deposit,USD,69.30,0.00,69.30,69.30,57L43782HY2956605,NaN,NaN,NaN,8729.0,0.0,0.0,299804319,47052051PE303031X


In [3]:
df.columns

Index(['Date', 'Time', 'Time Zone', 'Description', 'Currency', 'Gross', 'Fee',
       'Net', 'Balance', 'Transaction ID', 'From Email Address', 'Name',
       'Bank Name', 'Bank Account', 'Shipping and Handling Amount',
       'Sales Tax', 'Invoice ID', 'Reference Txn ID'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date                          238 non-null    object 
 1   Time                          238 non-null    object 
 2   Time Zone                     238 non-null    object 
 3   Description                   238 non-null    object 
 4   Currency                      238 non-null    object 
 5   Gross                         238 non-null    float64
 6   Fee                           238 non-null    float64
 7   Net                           238 non-null    float64
 8   Balance                       238 non-null    float64
 9   Transaction ID                238 non-null    object 
 10  From Email Address            108 non-null    object 
 11  Name                          112 non-null    object 
 12  Bank Name                     77 non-null     object 
 13  Bank 

In [5]:
drop_cols = ["Time Zone", "Currency", "Transaction ID", "Invoice ID", "Reference Txn ID"]
df = df.drop(drop_cols, axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date                          238 non-null    object 
 1   Time                          238 non-null    object 
 2   Description                   238 non-null    object 
 3   Gross                         238 non-null    float64
 4   Fee                           238 non-null    float64
 5   Net                           238 non-null    float64
 6   Balance                       238 non-null    float64
 7   From Email Address            108 non-null    object 
 8   Name                          112 non-null    object 
 9   Bank Name                     77 non-null     object 
 10  Bank Account                  78 non-null     float64
 11  Shipping and Handling Amount  238 non-null    float64
 12  Sales Tax                     238 non-null    float64
dtypes: fl

In [7]:
df[(df["From Email Address"].notna() & df["Name"].isna()) | \
   (df["From Email Address"].isna() & df["Name"].notna())]

,Date,Time,Description,Gross,Fee,Net,Balance,From Email Address,Name,Bank Name,Bank Account,Shipping and Handling Amount,Sales Tax
11,7/1/2020,01:21:01,BML Credit - Transfer from BML,12.99,0.0,12.99,0.00,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
13,7/1/2020,01:21:03,BML Credit - Transfer from BML,212.16,0.0,212.16,0.00,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
39,10/6/2020,11:55:32,External Hold,-19.11,0.0,-19.11,0.00,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
44,10/10/2020,02:11:57,External Release,19.11,0.0,19.11,19.11,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
46,10/20/2020,17:25:25,Tax collected by partner,-1.25,0.0,-1.25,38.19,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
56,11/10/2020,12:35:28,Tax collected by partner,-3.49,0.0,-3.49,53.00,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
57,11/10/2020,12:35:28,External Hold,-53.00,0.0,-53.00,0.00,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
62,11/14/2020,23:24:02,External Release,53.00,0.0,53.00,53.00,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
66,11/27/2020,07:22:54,Tax collected by partner,-1.50,0.0,-1.50,76.92,"To get contact details, please visit your Orde...",NaN,NaN,NaN,0.0,0.0
189,8/14/2022,20:25:41,Website Payment,-12.62,0.0,-12.62,-2.49,NaN,SQ *BLANK STREET,NaN,NaN,0.0,0.0


In [8]:
df[(df["From Email Address"].notna() & df["Name"].notna()) | \
   (df["From Email Address"].isna() & df["Name"].isna())]

,Date,Time,Description,Gross,Fee,Net,Balance,From Email Address,Name,Bank Name,Bank Account,Shipping and Handling Amount,Sales Tax
0,1/6/2020,21:58:32,PreApproved Payment Bill User Payment,-63.96,0.00,-63.96,-63.96,paypal_noa@nintendo.co.jp,Nintendo,NaN,NaN,0.0,0.0
1,1/6/2020,21:58:32,BML Credit - Transfer from BML,63.96,0.00,63.96,0.00,NaN,NaN,NaN,NaN,0.0,0.0
2,2/1/2020,00:14:41,General Payment,10.80,-0.78,10.02,10.02,480553751@qq.com,洁 刘,NaN,NaN,0.0,0.0
3,2/11/2020,15:36:27,General Withdrawal - Bank Account,-10.02,0.00,-10.02,0.00,NaN,NaN,PNC,2496.0,0.0,0.0
4,2/12/2020,10:30:37,General Card Deposit,69.30,0.00,69.30,69.30,NaN,NaN,NaN,8729.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,6/2/2023,09:17:49,General Card Deposit,92.57,0.00,92.57,0.00,NaN,NaN,"PNC Bank, National Association",8729.0,0.0,0.0
234,6/16/2023,10:29:39,PreApproved Payment Bill User Payment,-27.26,0.00,-27.26,-27.26,us-ebay-shiplabels18@ebay.com,eBay Inc Shipping,NaN,NaN,0.0,0.0
235,6/16/2023,10:29:39,Bank Deposit to PP Account,27.26,0.00,27.26,0.00,NaN,NaN,PNC,2496.0,0.0,0.0
236,6/29/2023,22:41:15,Express Checkout Payment,-55.00,0.00,-55.00,-55.00,BankingRequests@Conduent.com,NJ EZPASS TOLLS,NaN,NaN,0.0,0.0
